#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 2, 23)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market'.split()
colt = 'trans name qty target active spd current change percent reason market'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market
0,37,S,KCE,"4,000",66.00,0,CUT,SET50


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
3,B,MAKRO,-4,3L,SET,"3,000",42.00,43.00,-0.25,-0.58%,2,Buy,"126,000.00"
7,B,SCCC,-1,HD,SET,900,173.00,173.50,-0.50,-0.29%,2,Buy,"155,700.00"
10,B,SPALI,-6,3L,SET100,"9,000",22.00,22.60,0.20,+0.89%,2,Buy,"198,000.00"
15,S,GLOBAL,7,10pct,SET50,"5,000",21.80,21.10,0.00,0.00%,2,Sell,"109,000.00"
17,S,IVL,3,15pct,SET50,"1,200",48.25,47.50,0.25,+0.53%,2,Sell,"57,900.00"
19,S,LPF,2,DOS,SET,"10,000",14.10,13.90,0.10,+0.72%,2,Sell,"141,000.00"
24,S,RATCH,10,5pct,SET50,"3,000",48.00,45.50,-0.25,-0.55%,2,Sell,"144,000.00"
29,S,STA,10,CUT,SET100,"15,000",31.50,29.00,-0.25,-0.85%,2,Sell,"472,500.00"
34,S,WHART,4,DOS,SET,"10,000",12.80,12.40,0.10,+0.81%,2,Sell,"128,000.00"


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index().format(format_dict)

trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
B,MAKRO,-4,3L,SET,"3,000",42.00,43.00,-0.25,-0.58%,2,Buy,"126,000.00"
B,ASP,-7,6L,SET,"30,000",3.72,3.86,-0.02,-0.52%,1,Buy,"111,600.00"
B,KKP,-7,3L,SET100,"3,000",68.00,69.75,0.00,0.00%,1,Buy,"204,000.00"
B,NOBLE,-7,RD15%,SET,"21,000",5.85,6.20,0.00,0.00%,1,Buy,"122,850.00"
B,ASK,-21,1L,SET,"3,000",42.00,47.25,1.25,+2.72%,0,Buy,"126,000.00"


### End of Tables in the process

### Print to verify before upload file

In [84]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [85]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [86]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,MAKRO,"3,000",42.00,"126,000.00",3L,SET,2
1,B,TMT,"15,000",10.50,"157,500.00",ROUND,SET,2
2,S,GLOBAL,"5,000",21.80,"109,000.00",10pct,SET50,2
3,S,IVL,"1,200",48.25,"57,900.00",15pct,SET50,2
4,S,LPF,"10,000",14.10,"141,000.00",DOS,SET,2
5,S,SINGER,"1,500",48.50,"72,750.00",15pct,SET100,2
6,S,WHART,"10,000",12.80,"128,000.00",DOS,SET,2


In [87]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    283500.0
S    508650.0
Name: amount, dtype: float64

In [88]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,MAKRO,"3,000",42.00,"126,000.00",3L,SET,2
1,B,TMT,"15,000",10.50,"157,500.00",ROUND,SET,2


In [89]:
cash = 805_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(805000, 283500.0, 521500.0)

In [90]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
2,S,GLOBAL,"5,000",21.80,"109,000.00",10pct,SET50,2
3,S,IVL,"1,200",48.25,"57,900.00",15pct,SET50,2
4,S,LPF,"10,000",14.10,"141,000.00",DOS,SET,2
5,S,SINGER,"1,500",48.50,"72,750.00",15pct,SET100,2
6,S,WHART,"10,000",12.80,"128,000.00",DOS,SET,2


In [91]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(805000, 508650.0, 1313650.0)

### After call ord-log (must call everytime that orders change)

In [92]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [93]:
show(df[['trans','name','spd','reason','qty','target','amount','current','active']])

trans,name,spd,reason,qty,target,amount,current,active


In [94]:
mb = (df.trans == 'Buy') & (df.spd >= -5)
df[mb].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
3,B,MAKRO,-5,3L,SET,"3,000",42.00,43.25,0.00,0.00%,2,Buy,"126,000.00"
10,B,TMT,-5,ROUND,SET,"15,000",10.50,11.00,0.20,+1.85%,2,Buy,"157,500.00"


In [95]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [96]:
ms = (df.trans == 'Sell') & (df.spd <= 5)
df[ms].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
14,S,GLOBAL,4,10pct,SET50,"5,000",21.80,21.40,0.30,+1.42%,2,Sell,"109,000.00"
16,S,IVL,1,15pct,SET50,"1,200",48.25,48.00,0.75,+1.59%,2,Sell,"57,900.00"
18,S,LPF,2,DOS,SET,"10,000",14.10,13.90,0.10,+0.72%,2,Sell,"141,000.00"
20,S,MEGA,5,10pct,SET100,"1,500",47.25,46.00,3.25,+7.60%,1,Sell,"70,875.00"
27,S,SINGER,4,15pct,SET100,"1,500",48.50,47.50,3.00,+6.74%,2,Sell,"72,750.00"
33,S,WHART,3,DOS,SET,"10,000",12.80,12.50,0.20,+1.63%,2,Sell,"128,000.00"


In [97]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
3,B,MAKRO,-5,3L,SET,3000,42.00,43.25,0.00,0.00%,2,Buy,126000.0
10,B,TMT,-5,ROUND,SET,15000,10.50,11.00,0.20,+1.85%,2,Buy,157500.0
14,S,GLOBAL,4,10pct,SET50,5000,21.80,21.40,0.30,+1.42%,2,Sell,109000.0
16,S,IVL,1,15pct,SET50,1200,48.25,48.00,0.75,+1.59%,2,Sell,57900.0
18,S,LPF,2,DOS,SET,10000,14.10,13.90,0.10,+0.72%,2,Sell,141000.0
20,S,MEGA,5,10pct,SET100,1500,47.25,46.00,3.25,+7.60%,1,Sell,70875.0
27,S,SINGER,4,15pct,SET100,1500,48.50,47.50,3.00,+6.74%,2,Sell,72750.0
33,S,WHART,3,DOS,SET,10000,12.80,12.50,0.20,+1.63%,2,Sell,128000.0


In [98]:
df[mb | ms].shape[0]

8

In [99]:
mask = (df.trade == 'B')
df[mb | ms].nlargest(5, "amount")[["name","trans","amount"]].style.format(format_dict)

,name,trans,amount
10,TMT,Buy,"157,500.00"
18,LPF,Sell,"141,000.00"
33,WHART,Sell,"128,000.00"
3,MAKRO,Buy,"126,000.00"
14,GLOBAL,Sell,"109,000.00"


### Select source of orders between these two

In [100]:
df_out = df[mb | ms][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent,reason,market
3,Buy,MAKRO,"3,000",42.00,2,-5,43.25,0.00,0.00%,3L,SET
10,Buy,TMT,"15,000",10.50,2,-5,11.00,0.20,+1.85%,ROUND,SET
14,Sell,GLOBAL,"5,000",21.80,2,4,21.40,0.30,+1.42%,10pct,SET50
16,Sell,IVL,"1,200",48.25,2,1,48.00,0.75,+1.59%,15pct,SET50
18,Sell,LPF,"10,000",14.10,2,2,13.90,0.10,+0.72%,DOS,SET
20,Sell,MEGA,"1,500",47.25,1,5,46.00,3.25,+7.60%,10pct,SET100
27,Sell,SINGER,"1,500",48.50,2,4,47.50,3.00,+6.74%,15pct,SET100
33,Sell,WHART,"10,000",12.80,2,3,12.50,0.20,+1.63%,DOS,SET


In [56]:
df_out = df[df.active == 2][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent,reason,market
3,Buy,MAKRO,"3,000",42.00,2,-5,43.25,0.00,0.00%,3L,SET
7,Buy,SCCC,900,174.00,2,-2,175.00,1.00,+0.57%,HD,SET
14,Sell,GLOBAL,"5,000",21.80,2,4,21.40,0.30,+1.42%,10pct,SET50
16,Sell,IVL,"1,200",48.25,2,2,47.75,0.50,+1.06%,15pct,SET50
18,Sell,LPF,"10,000",14.10,2,2,13.90,0.10,+0.72%,DOS,SET
23,Sell,RATCH,"3,000",48.00,2,8,46.00,0.25,+0.55%,5pct,SET50
27,Sell,SINGER,"1,500",48.50,2,5,47.25,2.75,+6.18%,15pct,SET100
29,Sell,STA,"15,000",31.50,2,11,28.75,-0.50,-1.71%,CUT,SET100
33,Sell,WHART,"10,000",12.80,2,3,12.50,0.20,+1.63%,DOS,SET


In [101]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [102]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,MAKRO,"3,000",42.00,"126,000.00",3L,SET,2
1,B,TMT,"15,000",10.50,"157,500.00",ROUND,SET,2
2,S,GLOBAL,"5,000",21.80,"109,000.00",10pct,SET50,2
3,S,IVL,"1,200",48.25,"57,900.00",15pct,SET50,2
4,S,LPF,"10,000",14.10,"141,000.00",DOS,SET,2
5,S,SINGER,"1,500",48.50,"72,750.00",15pct,SET100,2
6,S,WHART,"10,000",12.80,"128,000.00",DOS,SET,2


### Evening Process

In [103]:
new_active = 1
old_active = 2

In [104]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

7

In [105]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'MAKRO', 'TMT', 'GLOBAL', 'IVL', 'LPF', 'MEGA', 'SINGER', 'WHART'"

In [106]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

8

In [107]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process